In [1]:
import re

import numpy as np
import keras
from keras.layers import Dense, Dropout, LSTM
from collections import defaultdict

In [2]:
text = open('./data/raw/text.txt', 'r', encoding='utf-8').read()
text = text.lower()
text = re.sub(r'[^a-zа-я\s.]+', '', text)
text = re.sub(r'\s+', ' ', text)

In [3]:
chars = sorted(list(set(text)))
char2int = dict()
int2char = dict()

for i, c in enumerate(chars):
    char2int[c] = i
    int2char[i] = c

In [4]:
window_length = 25
dataX = []
dataY = []
for sentence in text.split('.'):
  sentence += '.'
  for i in range(0, len(sentence) - window_length):
    dataX.append(sentence[i:i + window_length])
    dataY.append(sentence[i + window_length])

In [5]:
n_sentences = len(dataX)
X = np.zeros((n_sentences, window_length, len(chars)), dtype=np.bool_)
y = np.zeros((n_sentences, len(chars)), dtype=np.bool_)
for i, sentence in enumerate(dataX):
    for j, char in enumerate(sentence):
        X[i, j, char2int[char]] = True
    y[i, char2int[dataY[i]]] = True

In [6]:
model = keras.Sequential(
    [
        LSTM(256, input_shape=X.shape[1:], return_sequences=True),
        Dropout(0.2),
        LSTM(256),
        Dense(y.shape[1], activation='softmax')
    ]
)
model.compile(loss="categorical_crossentropy", optimizer='adam')
model.fit(X, y, batch_size=128, epochs=40)

2022-05-18 13:59:32.043154: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/40
85/85 [==============================] - 23s 220ms/step - loss: 3.2586
Epoch 2/40
85/85 [==============================] - 19s 224ms/step - loss: 3.1671
Epoch 3/40
85/85 [==============================] - 20s 234ms/step - loss: 3.0236
Epoch 4/40
85/85 [==============================] - 21s 252ms/step - loss: 2.7357
Epoch 5/40
85/85 [==============================] - 24s 287ms/step - loss: 2.4440
Epoch 6/40
85/85 [==============================] - 28s 333ms/step - loss: 2.2352
Epoch 7/40
85/85 [==============================] - 27s 313ms/step - loss: 2.0863
Epoch 8/40
85/85 [==============================] - 22s 261ms/step - loss: 1.9880
Epoch 9/40
85/85 [==============================] - 23s 268ms/step - loss: 1.9055
Epoch 10/40
85/85 [==============================] - 23s 266ms/step - loss: 1.8251
Epoch 11/40
85/85 [==============================] - 23s 264ms/step - loss: 1.7476
Epoch 12/40
85/85 [==============================] - 19s 223ms/step - loss: 1.6918
Epoch 13/40
8

In [10]:
index = np.random.randint(0, n_sentences - 1)
pattern = dataX[index]

print('Pattern = "' + pattern + '"')
for _ in range(100):
    x = np.zeros((1, window_length, len(chars)))
    for j, char in enumerate(pattern):
        x[0, j, char2int[char]] = 1.0
    preds = model.predict(x)
    next_char = int2char[np.argmax(preds)]
    pattern = pattern[1:] + next_char
    print(str(next_char), end="")

Pattern = " рэперских горлах взрывае"
тся порохом..я не дуцу прагоилая стибе икакак каракений.и..ри арий нарицару...ириши....ит.я ках тари

In [11]:
markov_window_length = 5
markov_dataX = []
markov_dataY = []
for sentence in text.split('.'):
  sentence += '.'
  for i in range(0, len(sentence) - markov_window_length):
    markov_dataX.append(sentence[i:i + markov_window_length])
    markov_dataY.append(sentence[i + markov_window_length])

nodes = defaultdict(lambda: defaultdict(lambda: 0))
for sentence, symbol in zip(markov_dataX, markov_dataY):
  nodes[sentence][symbol] += 1

In [22]:
index = np.random.randint(0, len(markov_dataX) - 1)

pattern = markov_dataX[index]
print('Pattern = "' + pattern + '"')
for i in range(100):
    next_chars_pool = [symbol for symbol in nodes[pattern]]
    preds = np.array([w for w in nodes[pattern].values()])
    preds = preds / preds.sum()
    if len(preds) == 0:
      break
    next_char = next_chars_pool[np.argmax(preds)]
    pattern = pattern[1:] + next_char
    print(str(next_char), end='')

Pattern = "бронз"
овый кейс прячет розовый кейс прячет розовый кейс прячет розовый кейс прячет розовый кейс прячет роз